In [ ]:
!py -3 -m pip install --upgrade Jinja2==3.0.3

In [ ]:
import sqlalchemy
import pandas as pd
import json
from jinjasql import JinjaSql

In [ ]:
with open('../Config/td_config.json') as json_td_config:
    config = json.load(json_td_config)

USER_NAME = config['user_name']
PASSWORD = config['pword']
HOST_NAME = config['host_name']

In [ ]:
td_engine = sqlalchemy.create_engine('teradatasql://' + HOST_NAME + '/?user=' + USER_NAME + '&password=' + PASSWORD + '&logmech=LDAP')

In [ ]:
_COLUMN_LIST_TEMPLATE = '''
SELECT TRIM(TRAILING ',' FROM (XMLAGG( TRIM(columnName) || ',' ORDER BY columnid) (VARCHAR(1000)))) uniq_col_list
FROM dbc.columnsV 
WHERE Lower(DatabaseName) = {{ database_name }} AND Lower(TableName) = {{ table_name }};
'''

In [ ]:
with open('../Config/table_config.json') as json_table_config:
    params = json.load(json_table_config)

In [ ]:
j = JinjaSql(param_style='qmark')
query, bind_params = j.prepare_query(_COLUMN_LIST_TEMPLATE, params)
print(query)
print(bind_params)

In [ ]:
col_list = pd.read_sql(query, td_engine,params=bind_params)
print(str(col_list['uniq_col_list'][0]))